In [ ]:

!pip install -q findspark
!pip install pyspark
!pip install openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=a09d6dbc19e6ceb4718b9e42e825517ee3d33c6fbabadd998c83cc5aaa13e6fc
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col
# Create a Spark session
spark = SparkSession.builder.appName("DataProcessingExample").getOrCreate()

# Load the dataset (replace "your_data.csv" with your actual file path)
df = spark.read.csv("cleaned_data.csv", header=True, inferSchema=True)

# Sample 10% of the data
sampled_df = df.sample(fraction=0.1, seed=42)

AnalysisException: ignored

In [ ]:
sampled_df.show()


+---------+--------+-----------+-----------+---------+----------+--------------+
|StockCode|Quantity|InvoiceDate|InvoiceTime|UnitPrice|CustomerID|       Country|
+---------+--------+-----------+-----------+---------+----------+--------------+
|    22633|       6| 2010/12/01|08:28:00 AM|     1.85|     17850|United Kingdom|
|    22622|       2| 2010/12/01|08:34:00 AM|     9.95|     13047|United Kingdom|
|    21755|       3| 2010/12/01|08:34:00 AM|     5.95|     13047|United Kingdom|
|    71053|       6| 2010/12/01|09:02:00 AM|     3.39|     17850|United Kingdom|
|    82486|       4| 2010/12/01|09:02:00 AM|     6.95|     17850|United Kingdom|
|    22752|       2| 2010/12/01|09:02:00 AM|     7.65|     17850|United Kingdom|
|   85123A|       6| 2010/12/01|09:32:00 AM|     2.55|     17850|United Kingdom|
|    21071|       6| 2010/12/01|09:32:00 AM|     1.06|     17850|United Kingdom|
|   84029E|       6| 2010/12/01|09:32:00 AM|     3.39|     17850|United Kingdom|
|    21033|      10| 2010/12

In [ ]:
# Select only the relevant columns
relevant_columns = ['CustomerID', 'StockCode', 'Quantity', 'UnitPrice']
df = df.select(relevant_columns)


In [ ]:
# Print the sampled DataFrame
df.show()


+----------+---------+--------+---------+
|CustomerID|StockCode|Quantity|UnitPrice|
+----------+---------+--------+---------+
|     17850|   85123A|       6|     2.55|
|     17850|    71053|       6|     3.39|
|     17850|   84406B|       8|     2.75|
|     17850|   84029G|       6|     3.39|
|     17850|   84029E|       6|     3.39|
|     17850|    22752|       2|     7.65|
|     17850|    21730|       6|     4.25|
|     17850|    22633|       6|     1.85|
|     17850|    22632|       6|     1.85|
|     13047|    84879|      32|     1.69|
|     13047|    22745|       6|      2.1|
|     13047|    22748|       6|      2.1|
|     13047|    22749|       8|     3.75|
|     13047|    22310|       6|     1.65|
|     13047|    84969|       6|     4.25|
|     13047|    22623|       3|     4.95|
|     13047|    22622|       2|     9.95|
|     13047|    21754|       3|     5.95|
|     13047|    21755|       3|     5.95|
|     13047|    21777|       4|     7.95|
+----------+---------+--------+---

In [ ]:
df = df.withColumn('Quantity', col('Quantity').cast('int'))

In [ ]:
# Group by 'CustomerID' and 'StockCode', then aggregate the sum of 'Quantity' and 'UnitPrice'
aggregated_df = df.groupBy('CustomerID', 'StockCode').agg(
    (sum('Quantity') * sum('UnitPrice')).alias('TotalQuantityTimesSales')
)


In [ ]:
aggregated_df.show()

+----------+---------+-----------------------+
|CustomerID|StockCode|TotalQuantityTimesSales|
+----------+---------+-----------------------+
|     18074|    22464|     19.799999999999997|
|     17908|    20966|                    2.5|
|     17908|   85099F|                   1.95|
|     17920|    22141|     25.200000000000003|
|     17968|    22500|                    9.9|
|     17897|    22909|                   0.85|
|     14729|    22582|                    5.1|
|     15012|    21041|                   2.95|
|     12433|    22665|     405.59999999999997|
|     15922|    22183|                   40.5|
|     17346|    22895|                   11.8|
|     17841|    22941|                  238.0|
|     13093|    20685|                  405.0|
|     14741|    22457|      619.5000000000001|
|     15658|    21756|                  17.85|
|     14060|    21014|                   51.0|
|     15363|    22382|                   16.5|
|     17964|    22745|                    2.1|
|     17235| 

In [ ]:
# Save DataFrame to CSV format
aggregated_df.write.csv("final1.csv", header=True, mode="overwrite")
